In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import argparse
import re
import time
from datetime import timedelta
from functools import partial
import logging
from datasets import load_dataset, Dataset
from tqdm import tqdm
import os
from torch.utils.data import DataLoader

from src.retriever import Retriever
from types import SimpleNamespace

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:

arg = SimpleNamespace()
arg.retrieval_model_name_or_path="facebook/contriever-msmarco"
arg.retrieval_embedding_size=768
arg.passages='data/corpus/psgs_w100.tsv'
arg.passages_embeddings='data/corpus/wikipedia_embeddings/*'
arg.indexing_batch_size=1000000
arg.save_or_load_index = False
arg.retrieval_n_subquantizers=0
arg.retrieval_n_bits=8
arg.max_k=100
arg.lowercase = False
arg.normalize_text = False
arg.per_gpu_batch_size=1000000
arg.question_maxlength=100000

retriever = Retriever(arg)
retriever.setup_retriever()

Loading model from: facebook/contriever-msmarco


/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Indexing passages from files ['data/corpus/wikipedia_embeddings/passages_00', 'data/corpus/wikipedia_embeddings/passages_01', 'data/corpus/wikipedia_embeddings/passages_02', 'data/corpus/wikipedia_embeddings/passages_03', 'data/corpus/wikipedia_embeddings/passages_04', 'data/corpus/wikipedia_embeddings/passages_05', 'data/corpus/wikipedia_embeddings/passages_06', 'data/corpus/wikipedia_embeddings/passages_07', 'data/corpus/wikipedia_embeddings/passages_08', 'data/corpus/wikipedia_embeddings/passages_09', 'data/corpus/wikipedia_embeddings/passages_10', 'data/corpus/wikipedia_embeddings/passages_11', 'data/corpus/wikipedia_embeddings/passages_12', 'data/corpus/wikipedia_embeddings/passages_13', 'data/corpus/wikipedia_embeddings/passages_14', 'data/corpus/wikipedia_embeddings/passages_15']
Loading file data/corpus/wikipedia_embeddings/passages_00
Total data indexed 1000000
Loading file data/corpus/wikipedia_embeddings/passages_01
Total data indexed 2000000
Loading file data/corpus/wikiped

In [4]:
#retriever.reset_args(arg)

In [5]:
from datasets import Dataset
dataset = Dataset.from_json("data/eval_data/triviaqa_test.jsonl")

In [20]:
idx = 0
print(dataset[idx]['question'])
print(dataset[idx]['answers'])
rs = retriever.search_document(['Hello Adele song','Led zepplin 4th album'], top_n=10)
print(rs[0])
print(rs[1])

Who was the man behind The Chipmunks?
['David Seville', 'david seville']
Questions embeddings shape: torch.Size([2, 768])


100%|██████████| 1/1 [00:05<00:00,  5.60s/it]

Search time: 5.6 s.
[{'id': '19100640', 'title': 'Hello (Adele song)', 'text': 'Hello (Adele song) "Hello" is a song by English singer-songwriter Adele, released on 23 October 2015 by XL Recordings as the lead single from her third studio album, "25" (2015). Adele co-wrote the song with her producer, Greg Kurstin. "Hello" is a piano ballad with soul influences, and lyrics that discuss themes of nostalgia and regret. Upon release, the song was acclaimed by music critics, who compared it favourably to Adele\'s previous work and praised the song\'s lyrics and Adele\'s vocals. It was recorded in London\'s Metropolis Studios. "Hello" reached number one in almost every country it charted in, including'}, {'id': '19100670', 'title': 'Hello (Adele song)', 'text': 'Dimes". Personnel Hello (Adele song) "Hello" is a song by English singer-songwriter Adele, released on 23 October 2015 by XL Recordings as the lead single from her third studio album, "25" (2015). Adele co-wrote the song with her pro

In [7]:
# import faiss

In [8]:
# res = faiss.StandardGpuResources()                          # 단일 GPU 사용하기

In [9]:
# retriever.index.index= faiss.index_cpu_to_gpu(res,0,retriever.index.index)

In [10]:
# x = retriever.embed_queries(retriever.args, ['a'+str(i) for i in range(100)])

In [11]:
# len(x)

In [12]:
# rs = retriever.search_document([str(i) for i in range(100)], top_n=100)
# print(len(rs))

In [13]:
# print(rs[59])

In [14]:

# questions_embedding = retriever.embed_queries(retriever.args,[str(100+i) for i in range(200)])

# # get top k results
# start_time_retrieval = time.time()
# top_ids_and_scores = retriever.index.search_knn(questions_embedding, retriever.args.max_k)

# print(f"Search time: {time.time()-start_time_retrieval:.1f} s.")

# final=[]
# for k in top_ids_and_scores:
#     tmp=[]
#     for kk in k[0]:
#         tmp.append(retriever.passage_id_map[kk])
#     final.append(tmp)
# # retriever.passage_id_map 
# # print(len(top_ids_and_scores))
# # print(len(top_ids_and_scores[0]))
# # print(len(top_ids_and_scores[0][0]))


In [15]:
# idx = 0
# print(dataset[idx]['question'])
# print(dataset[idx]['answers'])
# rs = retriever.search_document(['a'], top_n=10)
# rs = retriever.search_document(['a','b'], top_n=10)
# rs = retriever.search_document(['a','b','c'], top_n=10)

# rs = retriever.search_document(['a'+str(i) for i in range(100)], top_n=10)
# print(rs)

In [16]:
# print(len(rs))